# Feature engineering with pandas and scikit-learn

This notebook demonstrates how to use AI Platform notebooks to perform feature engineering on a dataset using `pandas`.

For each dataset, you will load the data into a `Pandas DataFrame`, clean up the columns into a usable format, and then restructure the data into feature and target data columns.

Before you jump in, let's cover some of the different tools you'll be using:

+ [AI Platform](https://cloud.google.com/ai-platform) consists of tools that allow machine learning developers and data scientists to run their ML projects quickly and cost-effectively.

+ [Cloud Storage](https://cloud.google.com/storage/) is a unified object storage for developers and enterprises, from live data serving to data analytics/ML to data archiving.

+ [Pandas](https://pandas.pydata.org/) is a data analysis and manipulation tool built on top of the Python programming language.

+ [scikit-learn](https://scikit-learn.org/stable/) is a machine learning and data analysis tool for the Python programming language that provides simple and efficient tools to analyze or predict data.

# Citibike Dataset

First, perform feature engineering on the [Citibike dataset](https://pantheon.corp.google.com/bigquery?project=data-science-onramp&p=bigquery-public-data&d=new_york_citibike&t=citibike_trips&page=table). This includes cleaning the data, extracting the necessary features, and transforming the data into feature columns.

## Load the data

### Import libraries
Running the following cell will import the libraries needed to preprocess the Citibike dataset.

+ `Pandas`: to store and manipulate the dataset
* `Google Cloud Storage`: to retrieve the dataset from the GCS bucket where the dataset is stored

In [2]:
from google.cloud import storage
import pandas as pd

### Define constants
Define the name of your Google Cloud Storage bucket where the cleaned data is stored.

+ `BLOB_PREFIX`: indicates the folder where the files are stored
+ `DIR_NAME`: holds the name of the local folder where the files will be downloaded to

In [3]:
BUCKET_NAME = 'your-bucket-name'
PROJECT_ID = 'your-project-id'
BLOB_PREFIX = 'clean_citibike_data2.csv.gz/part'
DIR_NAME = 'citibike_data'

### List the files

Run the following command to create a local folder where the dataset files will be stored.

In [4]:
!mkdir $DIR_NAME

Since the data cleaning job outputted multiple partioned files into the GCS bucket, you will need to loop through each file to access its contents. The following cell will retrieve all of the files with the `BLOB_PREFIX` defined above and download them. It will also create a list of the file names so they can be referenced later when loading the data into a dataframe.

In [5]:
# Create storage client
storage_client = storage.Client()

# List files in the bucket with the specified prefix
blobs = storage_client.list_blobs(BUCKET_NAME, prefix=BLOB_PREFIX)

# Go through the files and save them into the local folder
filenames = []
for i, blob in enumerate(blobs):
    filename = f'{DIR_NAME}/citibike{i}.csv.gz'
    blob.download_to_filename(filename)
    filenames.append(filename)
    print('Downloaded file: ' + str(blob.name))

Downloaded file: clean_citibike_data2.csv.gz/part-00000-01b8a983-e583-4863-a3d2-27185ec573c6-c000.csv.gz
Downloaded file: clean_citibike_data2.csv.gz/part-00001-01b8a983-e583-4863-a3d2-27185ec573c6-c000.csv.gz
Downloaded file: clean_citibike_data2.csv.gz/part-00002-01b8a983-e583-4863-a3d2-27185ec573c6-c000.csv.gz
Downloaded file: clean_citibike_data2.csv.gz/part-00003-01b8a983-e583-4863-a3d2-27185ec573c6-c000.csv.gz
Downloaded file: clean_citibike_data2.csv.gz/part-00004-01b8a983-e583-4863-a3d2-27185ec573c6-c000.csv.gz
Downloaded file: clean_citibike_data2.csv.gz/part-00005-01b8a983-e583-4863-a3d2-27185ec573c6-c000.csv.gz
Downloaded file: clean_citibike_data2.csv.gz/part-00006-01b8a983-e583-4863-a3d2-27185ec573c6-c000.csv.gz
Downloaded file: clean_citibike_data2.csv.gz/part-00007-01b8a983-e583-4863-a3d2-27185ec573c6-c000.csv.gz
Downloaded file: clean_citibike_data2.csv.gz/part-00008-01b8a983-e583-4863-a3d2-27185ec573c6-c000.csv.gz


### Load the files into a dataframe

Now, you can load the files into a dataframe.

First, define the schema. From this dataset, you will need 4 columns:

+ **starttime**: to extract the day of the week and date of when the trip starts
+ **stoptime**: to extract the day of the week and date of when the trip has ended
+ **start_station_id**: to find out how many trips started at a station
+ **end_station_id**: to find out how many trips ended at a station

In [6]:
COLUMNS = (
    'starttime',
    'stoptime',
    'start_station_id',
    'end_station_id',
)

Next, run the following cell to loop through the files downloaded to the local folder, create a `Pandas DataFrame`, and view the first ten rows.

In [7]:
# Create empty dataframe
citibike_data = pd.DataFrame()

# For each file: load the contents into a dataframe
# and concatenate the new dataframe with the existing
for file in filenames:
    print('Processing file: ' + file)
    new_df = pd.read_csv(file, compression='gzip', usecols=[1, 2, 3, 7], header=None,
                         names=COLUMNS, low_memory=False)
    citibike_data = pd.concat([citibike_data, new_df])

citibike_data.head(10)

Processing file: citibike_data/citibike0.csv.gz
Processing file: citibike_data/citibike1.csv.gz
Processing file: citibike_data/citibike2.csv.gz
Processing file: citibike_data/citibike3.csv.gz
Processing file: citibike_data/citibike4.csv.gz
Processing file: citibike_data/citibike5.csv.gz
Processing file: citibike_data/citibike6.csv.gz
Processing file: citibike_data/citibike7.csv.gz
Processing file: citibike_data/citibike8.csv.gz


,starttime,stoptime,start_station_id,end_station_id
0,2017-09-13T10:16:04,2017-09-13T10:31:17,3161.0,173.0
1,2015-09-28T14:39:07,2015-09-28T15:01:03,327.0,529.0
2,2014-07-03T14:48:15,2014-07-03T15:04:59,509.0,317.0
3,2015-08-01T19:19:46,2015-08-01T19:29:57,408.0,315.0
4,2016-03-06T10:45:06,2016-03-06T11:05:23,411.0,487.0
5,2017-11-01T08:49:57,2017-11-01T09:11:20,3535.0,3318.0
6,2018-05-08T19:00:14,2018-05-08T19:05:45,483.0,236.0
7,2018-05-16T08:51:27,2018-05-16T09:00:07,339.0,507.0
8,2017-11-15T10:26:11,2017-11-15T10:32:47,2003.0,527.0
9,2017-12-10T17:09:16,2017-12-10T17:13:07,229.0,285.0


## Extract features

### Clean up the data
The following cell will clean up the dataset in a few ways:

+ Any rows with NaN values will be dropped
+ The station IDs will be converted from floats to integers
+ The times from the start time column will be removed since they are not needed

In [8]:
# Drop rows with NaN values
citibike_data = citibike_data.dropna()

# Convert station IDs to integers
citibike_data['start_station_id'] = citibike_data['start_station_id'].astype('int32')
citibike_data['end_station_id'] = citibike_data['end_station_id'].astype('int32')

# Remove time from the time columns
citibike_data['starttime'] = citibike_data['starttime'].apply(lambda t: t.split("T")[0])
citibike_data['stoptime'] = citibike_data['stoptime'].apply(lambda t: t.split("T")[0])

citibike_data.head(10)

,starttime,stoptime,start_station_id,end_station_id
0,2017-09-13,2017-09-13,3161,173
1,2015-09-28,2015-09-28,327,529
2,2014-07-03,2014-07-03,509,317
3,2015-08-01,2015-08-01,408,315
4,2016-03-06,2016-03-06,411,487
5,2017-11-01,2017-11-01,3535,3318
6,2018-05-08,2018-05-08,483,236
7,2018-05-16,2018-05-16,339,507
8,2017-11-15,2017-11-15,2003,527
9,2017-12-10,2017-12-10,229,285


### Count trips starting from a station

Next, count the number of trips that have been taken from each station per day. The `groupby` function from `Pandas` will count the number of unique combinations of the start time and start station ID values. Then, the `pivot` function from `Pandas` can be used to convert the station IDs into columns (since they are the target data) and the counts as the values.

Also, use the `add_prefix` function to rename the columns and distinguish that the values indicate trips that have started from the station.

In [9]:
# Find unique combinations of start time and start station ID values
trips_started = (citibike_data.groupby(['starttime', 'start_station_id'])
                              .size().reset_index().rename(columns={0: 'count'}))

# Pivot to make station ID the columns and rename them
trips_started = (trips_started.pivot(index='starttime', columns='start_station_id', values='count')
                              .add_prefix('started_at_'))

trips_started.head(10)

start_station_id,started_at_72,started_at_79,started_at_82,started_at_83,started_at_116,started_at_119,started_at_120,started_at_127,started_at_128,started_at_137,...,started_at_3672,started_at_3673,started_at_3674,started_at_3675,started_at_3676,started_at_3680,started_at_3683,started_at_3684,started_at_3685,started_at_3686
starttime,,,,,,,,,,,,,,,,,,,,,
2013-07-01,55.0,88.0,18.0,30.0,56.0,NaN,11.0,103.0,83.0,9.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2013-07-02,83.0,120.0,27.0,26.0,78.0,2.0,12.0,105.0,123.0,10.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2013-07-03,84.0,112.0,26.0,42.0,83.0,2.0,19.0,141.0,109.0,10.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2013-07-04,85.0,126.0,38.0,60.0,81.0,NaN,9.0,138.0,104.0,13.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2013-07-05,80.0,135.0,31.0,35.0,67.0,3.0,6.0,84.0,93.0,22.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2013-07-06,84.0,167.0,40.0,41.0,60.0,2.0,21.0,107.0,102.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2013-07-07,87.0,145.0,17.0,44.0,69.0,7.0,25.0,93.0,109.0,18.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2013-07-08,109.0,143.0,29.0,47.0,97.0,4.0,9.0,112.0,124.0,9.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2013-07-09,122.0,134.0,25.0,50.0,110.0,3.0,19.0,135.0,150.0,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Count trips ending at a station

Running the following cell will repeat the same process as above, but will generate values for the number of trips that have ended at the station.

In [10]:
# Find unique combinations of start time and start station ID values
trips_ended = (citibike_data.groupby(['stoptime', 'end_station_id'])
                            .size().reset_index().rename(columns={0: 'count'}))

# Pivot to make station ID the columns and rename them
trips_ended = (trips_ended.pivot(index='stoptime', columns='end_station_id', values='count')
                          .add_prefix('ending_at_'))

trips_ended.head(10)

end_station_id,ending_at_72,ending_at_79,ending_at_82,ending_at_83,ending_at_116,ending_at_119,ending_at_120,ending_at_127,ending_at_128,ending_at_137,...,ending_at_3673,ending_at_3674,ending_at_3675,ending_at_3676,ending_at_3680,ending_at_3681,ending_at_3683,ending_at_3684,ending_at_3685,ending_at_3686
stoptime,,,,,,,,,,,,,,,,,,,,,
2013-07-01,60.0,82.0,10.0,31.0,54.0,NaN,9.0,93.0,78.0,7.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2013-07-02,84.0,122.0,26.0,29.0,76.0,2.0,15.0,97.0,125.0,13.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2013-07-03,87.0,121.0,28.0,48.0,73.0,3.0,16.0,131.0,109.0,18.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2013-07-04,85.0,144.0,39.0,55.0,77.0,2.0,16.0,129.0,127.0,18.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2013-07-05,85.0,134.0,34.0,42.0,63.0,4.0,7.0,75.0,95.0,27.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2013-07-06,73.0,152.0,36.0,58.0,58.0,1.0,27.0,86.0,103.0,10.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2013-07-07,70.0,159.0,25.0,49.0,68.0,7.0,24.0,60.0,128.0,19.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2013-07-08,109.0,137.0,31.0,57.0,92.0,7.0,13.0,105.0,119.0,12.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2013-07-09,136.0,149.0,22.0,65.0,102.0,4.0,22.0,117.0,153.0,8.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Putting it together

The following cell will combine both dataframes for trips started and ended at the stations. Then, the NaN values will be filled as 0's since this indicates that no trips started or ended at the particular stations.

In [11]:
# Combine the dataframes
# Set the index as row number instead of start time
# Fill the NaN values with 0's
citibike_df = (pd.concat([trips_started, trips_ended], axis=1)
               .reset_index()
               .fillna(0))

# Rename the column with start and end dates
citibike_df.rename(columns={'index': 'date'}, inplace=True)

citibike_df.head(10)

,date,started_at_72,started_at_79,started_at_82,started_at_83,started_at_116,started_at_119,started_at_120,started_at_127,started_at_128,...,ending_at_3673,ending_at_3674,ending_at_3675,ending_at_3676,ending_at_3680,ending_at_3681,ending_at_3683,ending_at_3684,ending_at_3685,ending_at_3686
0,2013-07-01,55.0,88.0,18.0,30.0,56.0,0.0,11.0,103.0,83.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2013-07-02,83.0,120.0,27.0,26.0,78.0,2.0,12.0,105.0,123.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2013-07-03,84.0,112.0,26.0,42.0,83.0,2.0,19.0,141.0,109.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2013-07-04,85.0,126.0,38.0,60.0,81.0,0.0,9.0,138.0,104.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2013-07-05,80.0,135.0,31.0,35.0,67.0,3.0,6.0,84.0,93.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,2013-07-06,84.0,167.0,40.0,41.0,60.0,2.0,21.0,107.0,102.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,2013-07-07,87.0,145.0,17.0,44.0,69.0,7.0,25.0,93.0,109.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,2013-07-08,109.0,143.0,29.0,47.0,97.0,4.0,9.0,112.0,124.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,2013-07-09,122.0,134.0,25.0,50.0,110.0,3.0,19.0,135.0,150.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,2013-07-10,86.0,140.0,41.0,39.0,139.0,4.0,24.0,166.0,131.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


You are done with feature engineering for the Citibike Dataset! Now, you can move on to the external datasets you ingested in BigQuery to obtain more features, starting with **Gas Prices**.

# Gas Prices Dataset

Now, perform feature engineering on the Gas Prices dataset. This includes cleaning the data, normalizing the price values, and transforming the data to match the Citibike dataset.

## Load the data

### Import libraries
Running the following cell will import the libraries needed to preprocess the Gas Prices dataset.

+ `BigQuery`: to retrieve the dataset from BigQuery
+ `scikit-learn`: to normalize the gas prices values

In [12]:
import datetime

from google.cloud import bigquery
from sklearn import preprocessing

### Load the data from BigQuery into a dataframe

Run the following cell to load the Gas Prices dataset from BigQuery into a dataframe. It will define a query to select the columns needed from the gas prices dataset, run the query using the BigQuery client, and then convert it to a `Pandas DataFrame`.

Make sure to fill in your project ID in the query.

In [13]:
LOCATION = "US"

# Create the BigQuery client
client = bigquery.Client(location=LOCATION)

# Define the query
table = f'{PROJECT_ID}.new_york_citibike_trips.gas_prices'
query = f' SELECT Date as date, New_York_City_Average_USD_per_Gal as nyc_gas_price FROM {table}'

# Run the query
query_job = client.query(
    query,
    location=LOCATION
)

# Convert to a dataframe
gas_df = query_job.to_dataframe()

gas_df.head(10)

,date,nyc_gas_price
0,12/15/2008,2.10
1,02/15/2016,2.08
2,12/28/2015,2.34
3,02/04/2008,3.27
4,12/20/2010,3.32
5,01/20/2014,3.72
6,12/23/2013,3.72
7,10/01/2012,4.17
8,03/02/2009,2.12
9,02/01/2016,2.17


## Normalize values

The gas price values range from around \\$2 to \\$5. It is important to normalize these values and scale them to be between 0 and 1 so that all the values within our dataset are weighted consistently. Running the following cell will create a scaler using the MinMaxScaler from scikit-learn and fit the gas prices to the scaler.

In [14]:
# Extract gas prices column as a numpy array
gas_values = gas_df[['nyc_gas_price']].values

# Create scaler from sklearn
min_max_scaler = preprocessing.MinMaxScaler()

# Fit values to the scaler and replace column with normalized values
gas_values_scaled = min_max_scaler.fit_transform(gas_values)
gas_df['nyc_gas_price'] = gas_values_scaled

gas_df.head(10)

,date,nyc_gas_price
0,12/15/2008,0.079681
1,02/15/2016,0.071713
2,12/28/2015,0.175299
3,02/04/2008,0.545817
4,12/20/2010,0.565737
5,01/20/2014,0.725100
6,12/23/2013,0.725100
7,10/01/2012,0.904383
8,03/02/2009,0.087649
9,02/01/2016,0.107570


## Copy prices for the week

The Citibike dataset contains values for each day, however, the Gas Prices dataset contains one value per week. To get values for each day, you can copy the price of the week's value for the entire seven days.

First, run the following cell to refactor the date so it matches the format of a datetime object.

In [15]:
def refactor_date(date):
    '''Refactor the date strings so they match the Citibike dataset'''
    parts = date.split('/')
    return f'{parts[2]}-{parts[0]}-{parts[1]}'


gas_df['date'] = gas_df['date'].apply(lambda d: refactor_date(d))
gas_df.head(10)

,date,nyc_gas_price
0,2008-12-15,0.079681
1,2016-02-15,0.071713
2,2015-12-28,0.175299
3,2008-02-04,0.545817
4,2010-12-20,0.565737
5,2014-01-20,0.725100
6,2013-12-23,0.725100
7,2012-10-01,0.904383
8,2009-03-02,0.087649
9,2016-02-01,0.107570


Now, copy the gas price of one day for the whole week by adding new rows to the dataframe.

The following cell does this by applying a function to each row in the dataframe that:
+ Converts each date to a datetime object
+ Loops through the next six days to create new rows
+ Appends the new rows to a list

In [16]:
# Define list to hold new rows
new_rows = []


def copy_values_for_week(row):
    '''Copies gas price of one day for the entire week '''
    today = datetime.datetime.strptime(row['date'], '%Y-%m-%d')
    # Loop through the next six days
    for day in range(1, 7):
        # Create and a new row for the next day
        new_day = datetime.datetime.strftime(today + datetime.timedelta(days=day), '%Y-%m-%d')
        new_row = {'date': new_day, 'nyc_gas_price': row['nyc_gas_price']}
        new_rows.append(new_row)

        
# Apply copy function to dataframe
gas_df.apply(copy_values_for_week, axis=1)

# Add new rows to dataframe
gas_df = gas_df.append(new_rows)

gas_df

,date,nyc_gas_price
0,2008-12-15,0.079681
1,2016-02-15,0.071713
2,2015-12-28,0.175299
3,2008-02-04,0.545817
4,2010-12-20,0.565737
...,...,...
3937,2011-05-04,0.924303
3938,2011-05-05,0.924303
3939,2011-05-06,0.924303
3940,2011-05-07,0.924303


You have now finished transforming the Gas Prices dataset! Now you can move on to the next external dataset: **US Holidays**.

# US Holidays Dataset

## Load the data

Run the following cell to load the US Holidays dataset from BigQuery into a dataframe. Similar to loading the Gas Prices dataset, this query selects the columns needed, runs the query using the BigQuery client, and converts the job to a dataframe.

In [17]:
# Define the query
table = f'{PROJECT_ID}.new_york_citibike_trips.usholidays'
query = f' SELECT Date as date, Holiday as holiday FROM {table}'

# Run the query
query_job = client.query(
    query,
    location=LOCATION,
)

# Convert to a dataframe
holiday_df = query_job.to_dataframe()

holiday_df

,date,holiday
0,1966-09-05,Labor Day
1,2010-09-06,Labor Day
2,1997-09-01,Labor Day
3,1993-09-06,Labor Day
4,1976-09-06,Labor Day
...,...,...
480,2013-01-21,"Birthday of Martin Luther King, Jr."
481,2020-01-20,"Birthday of Martin Luther King, Jr."
482,2015-01-19,"Birthday of Martin Luther King, Jr."
483,2012-01-16,"Birthday of Martin Luther King, Jr."


## Transform the holiday column

The purpose of the holiday feature column is to represent a binary value of whether there is a holiday on a specific day or not, rather than the type of holiday. Since this dataset contains only days with holidays, run the following cell to convert the holiday values to `True`. Later, when combining the datasets, you will add values of `False` to dates that are present in the other datasets but not this one.

In [18]:
holiday_df['holiday'] = holiday_df['holiday'].apply(lambda h: True)
holiday_df.head(10)

,date,holiday
0,1966-09-05,True
1,2010-09-06,True
2,1997-09-01,True
3,1993-09-06,True
4,1976-09-06,True
5,1982-09-06,True
6,2012-09-03,True
7,1981-09-07,True
8,1973-09-03,True
9,2016-09-05,True


You have now finished transforming the US Holidays dataset! Now you can move on to the next external dataset: **Weather**.

# Weather Dataset

## Load the data

Run the following cell to load the Weather dataset from BigQuery into a dataframe. For each year needed, you will:

+ Define a query that selects the required columns (whether there was impactful weather that day, precipitation (mm), minimum temperature, maximum temperature)
+ Run the query using the BigQuery client
+ Convert the job to a dataframe
+ Concatenate it with the combined dataframe

In [65]:
# Initialize combined weather dataframe
weather_df = pd.DataFrame()

years = ['2013', '2014', '2015', '2016', '2017', '2018']
for year in years:
    # Define a query
    query = (
        f''' SELECT
              date,
              IF(MAX(haswx) = 'True', 'True', 'False') AS impactful,
              MAX(prcp) AS prcp,
              MAX(tmin) AS min_temp,
              MAX(tmax) AS max_temp
             FROM (
              SELECT
                wx.date,
                IF (SUBSTR(wx.element, 0, 2) = 'WT', 'True', NULL) AS haswx,
                IF (wx.element = 'PRCP', wx.value/10, NULL) AS prcp,
                IF (wx.element = 'TMIN', wx.value/10, NULL) AS tmin,
                IF (wx.element = 'TMAX', wx.value/10, NULL) AS tmax
              FROM
                `bigquery-public-data.ghcn_d.ghcnd_{year}` AS wx
              WHERE
                id = 'USW00094728')
             GROUP BY
              date
             ORDER BY
              date'''
     )
    
    # Run the query
    query_job = client.query(
        query,
        location=LOCATION
    )

    # Convert to a dataframe
    curr_df = query_job.to_dataframe()
    
    # Concatenate with combined dataframe
    weather_df = pd.concat([weather_df, curr_df])

weather_df.head(10)  

,date,impactful,prcp,min_temp,max_temp
0,2013-01-01,False,0.0,-3.3,4.4
1,2013-01-02,True,0.0,-5.6,0.6
2,2013-01-03,False,0.0,-4.4,0.0
3,2013-01-04,False,0.0,-1.1,2.8
4,2013-01-05,False,0.0,0.0,5.6
5,2013-01-06,True,0.0,1.1,7.8
6,2013-01-07,False,0.0,2.8,7.2
7,2013-01-08,False,0.0,1.7,8.9
8,2013-01-09,True,0.0,3.9,9.4
9,2013-01-10,False,0.0,4.4,8.3


## Add average temperature

In addition to the minimum and maximum temperatures, it would be helpful to have an average temperature value. Run the following cell to apply a function to each row and create an *avg_temp* column.

In [66]:
def get_average_temp(row):
    ''' Find the average temperature per row '''
    return (row['min_temp'] + row['max_temp']) / 2

weather_df['avg_temp'] = weather_df.apply(get_average_temp, axis=1)
weather_df.head(10)

,date,impactful,prcp,min_temp,max_temp,avg_temp
0,2013-01-01,False,0.0,-3.3,4.4,0.55
1,2013-01-02,True,0.0,-5.6,0.6,-2.50
2,2013-01-03,False,0.0,-4.4,0.0,-2.20
3,2013-01-04,False,0.0,-1.1,2.8,0.85
4,2013-01-05,False,0.0,0.0,5.6,2.80
5,2013-01-06,True,0.0,1.1,7.8,4.45
6,2013-01-07,False,0.0,2.8,7.2,5.00
7,2013-01-08,False,0.0,1.7,8.9,5.30
8,2013-01-09,True,0.0,3.9,9.4,6.65
9,2013-01-10,False,0.0,4.4,8.3,6.35


## Normalize Values

Similar to the gas price values, the precipitation and temperature values must be normalized so all the values within the dataset are weighted consistently.

In [67]:
cols_to_normalize = ['prcp', 'min_temp', 'max_temp', 'avg_temp']

for col_name in cols_to_normalize:
    # Extract values
    temp_values = weather_df[[col_name]].values

    # Fit values to the scaler and replace column with normalized values
    temp_values_scaled = min_max_scaler.fit_transform(temp_values)
    weather_df[col_name] = temp_values_scaled

weather_df.head(10)

,date,impactful,prcp,min_temp,max_temp,avg_temp
0,2013-01-01,False,0.0,0.320430,0.315678,0.312910
1,2013-01-02,True,0.0,0.270968,0.235169,0.246171
2,2013-01-03,False,0.0,0.296774,0.222458,0.252735
3,2013-01-04,False,0.0,0.367742,0.281780,0.319475
4,2013-01-05,False,0.0,0.391398,0.341102,0.362144
5,2013-01-06,True,0.0,0.415054,0.387712,0.398249
6,2013-01-07,False,0.0,0.451613,0.375000,0.410284
7,2013-01-08,False,0.0,0.427957,0.411017,0.416849
8,2013-01-09,True,0.0,0.475269,0.421610,0.446389
9,2013-01-10,False,0.0,0.486022,0.398305,0.439825


## Convert the *date* column's data type

Run the following cell to change the datatype of the date column from a Datetime object to a string, so it can be properly combined with the other datasets.

In [69]:
weather_df['date'] = weather_df['date'].apply(lambda d: datetime.datetime.strftime(d, '%Y-%m-%d'))
weather_df.head(10)

,date,impactful,prcp,min_temp,max_temp,avg_temp
0,2013-01-01,False,0.0,0.320430,0.315678,0.312910
1,2013-01-02,True,0.0,0.270968,0.235169,0.246171
2,2013-01-03,False,0.0,0.296774,0.222458,0.252735
3,2013-01-04,False,0.0,0.367742,0.281780,0.319475
4,2013-01-05,False,0.0,0.391398,0.341102,0.362144
5,2013-01-06,True,0.0,0.415054,0.387712,0.398249
6,2013-01-07,False,0.0,0.451613,0.375000,0.410284
7,2013-01-08,False,0.0,0.427957,0.411017,0.416849
8,2013-01-09,True,0.0,0.475269,0.421610,0.446389
9,2013-01-10,False,0.0,0.486022,0.398305,0.439825


# Combine the datasets

Now that all of the datasets have been transformed, you can combine them to create one table. Run the following cell to combine the dataframes.

+ **Citibike Trips** and **Gas Prices**: merge on the date column by specifying `on="date"` to create a *combined dataframe*
+ *Combined dataframe* and **US Holidays**: merge both datasets on the date column, keep the dates of the combined dataframe by specifying `how='right'`, and fill the empty rows with `False`
+ *Combined dataframe* and **Weather**: merge on the date column by specifying `on="date"` to create the *final dataframe*

In [70]:
# Merge both gas dataset with citibike dataset
final_df = pd.merge(gas_df, citibike_df, on="date")

# Merge combined dataset with holiday dataset
final_df = pd.merge(holiday_df, final_df, how="right", on="date").fillna(False)

# Merge combined dataset with weather dataset
final_df = pd.merge(weather_df, final_df, on="date")

final_df

,date,impactful,prcp,min_temp,max_temp,avg_temp,holiday,nyc_gas_price,started_at_72,started_at_79,...,ending_at_3673,ending_at_3674,ending_at_3675,ending_at_3676,ending_at_3680,ending_at_3681,ending_at_3683,ending_at_3684,ending_at_3685,ending_at_3686
0,2013-07-01,False,0.168780,0.868817,0.752119,0.817287,False,0.768924,55.0,88.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2013-07-02,True,0.015848,0.868817,0.811441,0.847921,False,0.768924,83.0,120.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2013-07-03,True,0.106973,0.881720,0.822034,0.859956,False,0.768924,84.0,112.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2013-07-04,True,0.000000,0.905376,0.870763,0.897155,True,0.768924,85.0,126.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2013-07-05,False,0.000000,0.916129,0.904661,0.920131,False,0.768924,80.0,135.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1660,2018-08-12,True,0.000000,0.868817,0.834746,0.859956,False,0.478088,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1661,2018-08-21,True,0.022187,0.797849,0.775424,0.793217,False,0.482072,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1662,2018-08-25,False,0.000000,0.821505,0.788136,0.811816,False,0.482072,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1663,2018-09-01,False,0.000000,0.834409,0.764831,0.806346,False,0.478088,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Improve the date feature

Now that all the datasets have been combined, you can separate the start time column into more features such as the year, month, and day. Then, the date column can be dropped.

In [71]:
# Define the name and year, month, and day columns
date_columns = final_df['date'].str.split('-', expand=True)
date_names = ['year', 'month', 'day']

# Add the columns at the start of the dataset
for i in range(3):
    final_df.insert(0, date_names[i], date_columns[i])
    final_df[date_names[i]] = final_df[date_names[i]].astype('int32')

# Remove the date column from the dataframe
final_df = final_df.drop('date', axis=1)

final_df.head(10)

,day,month,year,impactful,prcp,min_temp,max_temp,avg_temp,holiday,nyc_gas_price,...,ending_at_3673,ending_at_3674,ending_at_3675,ending_at_3676,ending_at_3680,ending_at_3681,ending_at_3683,ending_at_3684,ending_at_3685,ending_at_3686
0,1,7,2013,False,0.168780,0.868817,0.752119,0.817287,False,0.768924,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,7,2013,True,0.015848,0.868817,0.811441,0.847921,False,0.768924,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3,7,2013,True,0.106973,0.881720,0.822034,0.859956,False,0.768924,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4,7,2013,True,0.000000,0.905376,0.870763,0.897155,True,0.768924,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5,7,2013,False,0.000000,0.916129,0.904661,0.920131,False,0.768924,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,6,7,2013,False,0.000000,0.941935,0.927966,0.945295,False,0.768924,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,7,7,2013,False,0.000000,0.941935,0.927966,0.945295,False,0.768924,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,8,7,2013,False,0.044374,0.881720,0.894068,0.897155,False,0.764940,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,9,7,2013,False,0.045959,0.892473,0.881356,0.896061,False,0.764940,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,10,7,2013,False,0.000000,0.905376,0.845339,0.884026,False,0.764940,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


The following cell will extract the day of the week from the date information using the `Datetime` python library.

In [72]:
def find_weekday(df):
    ''' Creates a datetime object and returns the day of the week '''
    date = datetime.datetime(int(df['year']), int(df['month']), int(df['day']))
    return date.weekday()


# Apply the find_weekday() function to every row of the dataset
weekday_col = final_df.apply(find_weekday, axis=1)

# Insert the weekday column at the start
final_df.insert(0, 'weekday', weekday_col)

final_df

,weekday,day,month,year,impactful,prcp,min_temp,max_temp,avg_temp,holiday,...,ending_at_3673,ending_at_3674,ending_at_3675,ending_at_3676,ending_at_3680,ending_at_3681,ending_at_3683,ending_at_3684,ending_at_3685,ending_at_3686
0,0,1,7,2013,False,0.168780,0.868817,0.752119,0.817287,False,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,2,7,2013,True,0.015848,0.868817,0.811441,0.847921,False,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,3,7,2013,True,0.106973,0.881720,0.822034,0.859956,False,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3,4,7,2013,True,0.000000,0.905376,0.870763,0.897155,True,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,5,7,2013,False,0.000000,0.916129,0.904661,0.920131,False,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1660,6,12,8,2018,True,0.000000,0.868817,0.834746,0.859956,False,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1661,1,21,8,2018,True,0.022187,0.797849,0.775424,0.793217,False,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1662,5,25,8,2018,False,0.000000,0.821505,0.788136,0.811816,False,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1663,5,1,9,2018,False,0.000000,0.834409,0.764831,0.806346,False,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
